# Digital Twin model

## Initialization

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from ML.DTPredictor import DTPredictor
from PreProcessing.DataManipulation import DataManipulation
import pickle

In [12]:
dm = DataManipulation("../data/deep-scenario.csv")
dm.data.head(3)

,Execution,ScenarioID,Configuration_API_Description,Attribute[TTC],Attribute[DTO],Attribute[Jerk],Attribute[COL],Attribute[COLT],Attribute[SAC],reward,road,strategy,scenario
0,0,0_scenario_0,A red BoxTruck is overtaking (near) the ego ve...,100000.000000,24.810964,3.48,False,None,0.0,dto,road1,greedy,rain_day
1,0,0_scenario_1,A skyblue SUV is driving from the opposite dir...,100000.000000,8.654072,1.04,False,None,0.0,dto,road1,greedy,rain_day
2,0,0_scenario_2,A skyblue BoxTruck is driving ahead (near) of ...,2.103269,10.209357,4.98,False,None,0.0,dto,road1,greedy,rain_day


## Data Manipulation

In [13]:
dm.addFromXML("../data/dataFromXML.csv")

In [14]:
ids = [268, 273, 477, 1406, 1662, 1961, 2000, 2001, 2002, 2091]
cols = ["Attribute[DTO]", "speed1", "speed2", "speed3", "speed4", "speed5", "speed6", "Attribute[SAC]"]
# dm.data.loc[dm.data["Attribute[COL]"] == True][["Attribute[DTO]", "speed1", "speed2", "speed3", "speed4", "speed5", "speed6", "Attribute[SAC]"]]
dm.data.iloc[ids][cols]

,Attribute[DTO],speed1,speed2,speed3,speed4,speed5,speed6,Attribute[SAC]
268,1.295431,2.623626,3.582205,3.469742,2.785889,3.443,4.764,4.442793
273,1.331153,1.536797,1.481917,1.723328,1.945000,3.981,5.591,2.346909
477,1.411721,1.735009,0.914792,1.804000,2.700000,3.077,4.397,2.378859
1406,4.095299,2.743850,3.850416,3.298107,3.749705,3.705,4.302,3.715036
1662,1.378684,1.628659,2.188346,2.626000,3.734000,5.172,5.616,3.522738
1961,1.473800,5.087671,4.348258,5.423379,5.005000,6.832,6.982,5.225631
2000,50.289233,0.082000,1.422000,2.504000,0.007000,0.003,0.001,1.340647
2001,25.204523,0.135000,1.481000,2.582000,0.003000,0.002,0.005,0.217742
2002,27.160148,0.135000,1.557000,2.778000,0.003000,0.005,0.004,0.344920
2091,1.511598,3.467284,2.311596,3.271000,4.587000,6.061,7.418,3.923682


In [15]:
# dm.underSample(1000)

## Model

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [17]:
dm.underSample(3000)
dm.data.value_counts("Attribute[COL]")

Attribute[COL]
False    3000
True     1050
dtype: int64

In [18]:
trainX, trainY, testX, testY = dm.splitTrainTest()
print(f"trainX:{trainX.shape}, trainY:{trainY.shape}, testX:{testX.shape}, testY:{testY.shape}")

splitting at 3240.
trainX:(3240, 30), trainY:(3240, 4), testX:(810, 30), testY:(810, 4)


### Random Forest

In [19]:
rf_clf = DTPredictor(RandomForestClassifier(n_estimators=100, max_depth=10, random_state=1))
rf_trainX, rf_trainY = rf_clf.preProcess(trainX.copy(), trainY.copy())
rf_testX, rf_testY = rf_clf.preProcess(testX.copy(), testY.copy())

rf_clf.fit(rf_trainX, rf_trainY)

Scaler is fitted


In [20]:
rf_pred = rf_clf.predict(rf_testX)
rf_score = rf_clf.getScore(rf_testY, rf_pred)

Total: 810, Collisions: 217
Accuracy: 0.9493827160493827, Precision: 0.9444444444444444, Recall: 0.8617511520737328, F1: 0.9012048192771084
Confusion matrix:
[[582  11]
 [ 30 187]]


### Support Vector Machine - Classifier

In [21]:
svm_clf = DTPredictor(SVC(random_state=1))
svm_trainX, svm_trainY = svm_clf.preProcess(trainX.copy(), trainY.copy())
svm_testX, svm_testY = svm_clf.preProcess(testX.copy(), testY.copy())

Scaler is fitted


In [22]:
svm_clf.fit(svm_trainX, svm_trainY)

In [23]:
svm_pred = svm_clf.predict(svm_testX)
svm_score = svm_clf.getScore(svm_testY, svm_pred)

Total: 810, Collisions: 217
Accuracy: 0.8962962962962963, Precision: 0.9182389937106918, Recall: 0.6728110599078341, F1: 0.776595744680851
Confusion matrix:
[[580  13]
 [ 71 146]]


### xgboost - Classifier

In [24]:
xgb_clf = DTPredictor(XGBClassifier(n_estimators=100, random_state=1))
xgb_trainX, xgb_trainY = xgb_clf.preProcess(trainX.copy(), trainY.copy())
xgb_testX, xgb_testY = xgb_clf.preProcess(testX.copy(), testY.copy())

Scaler is fitted


In [25]:
xgb_clf.fit(xgb_trainX, xgb_trainY)

In [26]:
xgb_pred = xgb_clf.predict(xgb_testX)
xgb_score = xgb_clf.getScore(xgb_testY, xgb_pred)

Total: 810, Collisions: 217
Accuracy: 0.9617283950617284, Precision: 0.9514563106796117, Recall: 0.9032258064516129, F1: 0.9267139479905436
Confusion matrix:
[[583  10]
 [ 21 196]]


In [30]:
# model_score = f"{score[0][0]}-{score[0][1]}-{score[1][0]}-{score[1][1]}"
model_score = f"{xgb_score[0][0]}-{xgb_score[0][1]}-{xgb_score[1][0]}-{xgb_score[1][1]}"
model_score

'583-10-21-196'

In [29]:
xgb_clf.saveModel("xgb", accuracy=model_score)

Model saved!
